# Book for Saving/Loading Bandpower and Network Features

## imports

In [1]:
from glob import glob
import os
import sys

code_path = os.path.dirname(os.getcwd())
sys.path.append(code_path)

import warnings
import json
from os.path import join as ospj
import numpy as np
import pandas as pd
import tools
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

os.environ['PYTHONWARNINGS']='ignore::FutureWarning' # for parallel

warnings.filterwarnings("ignore")

from sklearn import *

### set Params

In [2]:
bands = [
    [0.5, 4], # delta
    [4, 8], # theta
    [8, 12], # alpha
    [12, 30], # beta
    [30, 80], # gamma
    [0.5, 80] # broad
]
band_names = ["delta", "theta", "alpha", "beta", "gamma", "broad"]
N_BANDS = len(bands)

### load metadata

In [3]:
with open("../config.json", 'rb') as f:
    config = json.load(f)
repo_path = config['repositoryPath']
metadata_path = ospj(repo_path, 'ieeg-metadata')
data_path = ospj(repo_path, 'data')

In [4]:
patient_list = pd.read_csv(ospj(metadata_path, "patient_list.csv"))['patient']


## Main

### Load patient bandpower

In [6]:
AllPtPxx = []
ZScoreDf = []
time_periods = ['interictal']
for pt_num, patient in tqdm(enumerate(patient_list), total=len(patient_list)):
    try:
        PtDf  = tools.LoadBandpower(data_path, patient, time_periods)
    except:
        continue
    
    AllPtPxx.append(PtDf)
AllPtPxx = pd.concat(AllPtPxx)
AllPtPxx.to_pickle(ospj(data_path, "AllPtPxxBipolar.pkl"))

100%|██████████| 101/101 [00:22<00:00,  4.54it/s]


### Load Networks and Calculate Features

In [7]:
AllPtNxx = []
time_periods = ['interictal']
for pt_num, patient in tqdm(enumerate(patient_list), total=len(patient_list)):
    try:
        PtDf  = tools.LoadNetworks(data_path, patient, time_periods)
    except:
        continue
    AllPtNxx.append(PtDf)
AllPtNxx = pd.concat(AllPtNxx)
AllPtNxx = (
        AllPtNxx.melt(ignore_index=False, var_name="band", value_name="coherence")
        .set_index("band", append=True)
        .reorder_levels(["patient", "period", "clip",  'band', "channel_1", "channel_2",])
    )
AllPtNxx.to_pickle(ospj(data_path, "AllPtNxxBipolar.pkl"))

100%|██████████| 101/101 [00:14<00:00,  6.83it/s]
